In [13]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr
import pytz
from datetime import datetime as dt,timedelta
import sys,os,glob
import matplotlib.image as mpimg

In [12]:
## Graphical parameters
plt.style.use(os.path.join(matplotlib.get_configdir(),'stylelib/presentation.mplstyle'))

In [2]:
indir_mtpw = '/Users/bfildier/Data/satellite/MIMIC-TPW2/data/'
indir_goespw = '/Users/bfildier/Data/satellite/GOES-PW/'
indir_sonde_qrad = '/Users/bfildier/Data/EUREC4A/merged/radiative_profiles'

In [3]:
scriptdir = os.getcwd()
repodir = os.path.dirname(scriptdir)
figdir = os.path.join(repodir,'figures')
os.makedirs(figdir,exist_ok=True)

In [15]:
## Own modules

moduledir = os.path.join(repodir,'functions')

sys.path.insert(0,moduledir)
print("Own modules available:", [os.path.splitext(os.path.basename(x))[0]
                                 for x in glob.glob(os.path.join(moduledir,'*.py'))])

from matrixoperators import *

Own modules available: ['conditionalstats', 'radiativefeatures', 'matrixoperators']


In [18]:
day = '20200126'
date = pytz.utc.localize(dt.strptime(day,'%Y%m%d'))

lat_box = 11,16
lon_box = -60,-52

dim_t = 0
dim_z = 1

# Load data

## Sonde profiles

In [5]:
print('load rad profiles')
radprf = xr.open_dataset(os.path.join(indir_sonde_qrad,'rad_profiles_CF.nc'))

load rad profiles


In [6]:
radprf_day = radprf.where(radprf.z_min<=50,drop=True).sel(launch_time=day)

In [22]:
print('compute PW')

mo = MatrixOperators()

QV_prf = radprf_day.specific_humidity.data # kg(w)/kg(a)
pres = np.nanmean(radprf_day.pressure.data,axis=dim_t)*100 # hPa
PW_prf = mo.pressureIntegral(QV_prf,pres,z_axis=dim_z)

compute PW


/Users/bfildier/anaconda3/envs/pyLMD/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: Mean of empty slice
  


## GOES-TPW

In [25]:
goes_files = glob.glob(os.path.join(indir_goespw,day,'*.nc'))
goes_files.sort()

In [26]:
print('load and concatenate all GOES TPW files for that day')

goes_all = []

for file in goes_files:

    data_new = xr.open_dataset(file)
    goes_all.append(data_new)


load and concatenate all GOES TPW files for that day


In [28]:
## concatenate
data_goes = xr.concat(goes_all,dim='t')

In [29]:
pw_goes = data_goes.TPW.data

## Compute GOES PW at sonde locations

In [33]:
print("Get coordinates of GOES PW values")

##-- code structures found here: https://github.com/blaylockbk/pyBKB_v2/blob/master/BB_goes16/mapping_GOES16_data.ipynb
##-- detail on all angles: https://makersportal.com/blog/2018/11/25/goes-r-satellite-latitude-and-longitude-grid-projection-algorithm

from mpl_toolkits.basemap import Basemap
from pyproj import Proj

##-- satellite info

# Satellite height
sat_h = data_goes.nominal_satellite_height[0].data*1e3 # in km

# Satellite longitude
sat_lon = data_goes.nominal_satellite_subpoint_lon[0].data

##-- longitude and latitude

p = Proj(proj='geos', h=sat_h, lon_0=sat_lon, sweep='x')

# The projection x and y coordinates equals
# the scanning angle (in radians) multiplied by the satellite height (http://proj4.org/projections/geos.html)
X = data_goes.variables['x'][:] * sat_h
Y = data_goes.variables['y'][:] * sat_h
T = data_goes.variables['t'][:]

XX, YY, TT = np.meshgrid(X, Y, T)
goes_lons, goes_lats = p(XX, YY, inverse=True)

lons[np.isinf(lons)] = np.nan
lats[np.isinf(lats)] = np.nan

# mask_valid = np.logical_not(np.isnan(pw_goes))
# lons_goes_1D = lons[mask_valid]
# lats_goes_1D = lats[mask_valid]
# pw_goes_1D = pw_goes[mask_valid]

Get coordinates of GOES PW values


In [36]:
date

datetime.datetime(2020, 1, 26, 0, 0, tzinfo=<UTC>)

In [62]:
print(data_goes.variables['t'][0])
print(np.datetime64(day))
ts = (data_goes.variables['t'][0] - np.datetime64(day)) / np.timedelta64(1, 's')
dt.utcfromtimestamp(ts)

<xarray.Variable ()>
array('2020-01-26T00:05:01.116525952', dtype='datetime64[ns]')
Attributes:
    long_name:      time variable (t) is the mid-point between the start and ...
    standard_name:  time
    axis:           T
    bounds:         time_bounds
20200126


datetime.datetime(1960, 2, 18, 14, 39, 48)

In [63]:
np.datetime64(day)

numpy.datetime64('20200126')